In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import pymongo

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\breck\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [4]:
# URL of page to be scraped
url = "https://redplanetscience.com/"
browser.visit(url)

In [5]:
html = browser.html

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

## Step 1 - Scraping

### NASA Mars News

In [8]:
#Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
news_title = soup.find_all('div', class_="content_title")[0].text.strip()
print(news_title)

Mars Is Getting a New Robotic Meteorologist


In [9]:
news_p = soup.find_all('div', class_="article_teaser_body")[0].text.strip()
print(news_p)

Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.


### JPL Mars Space Images - Feautured Image

In [10]:
executable_path2 = {'executable_path': ChromeDriverManager().install()}
browser2 = Browser('chrome', **executable_path2, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\breck\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [11]:
img_url = 'https://spaceimages-mars.com/'
browser2.visit(img_url)

In [12]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
browser2.links.find_by_partial_text('FULL IMAGE').click()

In [13]:
html2 = browser2.html
image_soup = BeautifulSoup(html2, "html.parser")

In [14]:
relative_image_path = image_soup.find('img', class_='fancybox-image')["src"]
featured_img_url = img_url + relative_image_path
featured_img_url


'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [15]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
facts_url = 'https://galaxyfacts-mars.com'
facts_table = pd.read_html(facts_url)[1]
facts_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
#Use Pandas to convert the data to a HTML table string.
facts_table = facts_table.to_html()

### Mars Hemispheres

In [17]:
# Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
executable_path3 = {'executable_path': ChromeDriverManager().install()}
browser3 = Browser('chrome', **executable_path3, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\breck\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [18]:
hemi_url = 'https://marshemispheres.com/'
browser3.visit(hemi_url)

In [19]:
hemisphere_image_urls = []

for hemi in range(4):
    browser3.links.find_by_partial_text('Hemisphere')[hemi].click()
    
    html3 = browser3.html
    hemi_soup = BeautifulSoup(html3, "html.parser")

    title = hemi_soup.find('h2', class_='title').text.strip()
    img_url = hemi_soup.find('li').a.get('href')
    
    hemis = {}
    hemis['title'] = title
    hemis['img_url'] = f'https://marshemispheres.com/{img_url}'
    hemisphere_image_urls.append(hemis)

    browser3.back()


In [20]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [28]:
mars_data = {
    'news_title': news_title,
    'news_paragraph': news_p,
    'featured_img_url': featured_img_url,
    'facts_table': facts_table,
    'hemispheres': hemisphere_image_urls
    }
mars_data

{'news_title': 'Mars Is Getting a New Robotic Meteorologist',
 'news_paragraph': "Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.",
 'featured_img_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
 'facts_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n 

##Step 2 - MongoDB and Flask Application

In [26]:

#Create a connection to 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [27]:
#Create a MongoDB database
db = client.mars_db

#Drop database if existing
db.mars.drop()

#Create collection
#data_collection = db["mars"]

#Insert dictionary into mars_db
db.mars.insert_one(mars_data)